説明を読む感じでは、これはLangChain側で実装してもいいかなって感じはします
https://medium.com/@thomas_reid/llamaparse-rag-beats-all-comers-60948c6cc0e4

この記事を参考にしたコードを作成する
Rerankingと複数のpdfを読み込めるかをテストしながら実装してみる

In [2]:
import nest_asyncio
nest_asyncio.apply()
from dotenv import load_dotenv
import logging
import sys

# ログレベルの設定
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG, force=True)

load_dotenv()

True

In [3]:
from llama_index.core import VectorStoreIndex
from llama_index.llms.openai import OpenAI
from llama_parse import LlamaParse
from llama_index.core import Settings
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.node_parser import MarkdownElementNodeParser
from llama_index.postprocessor.cohere_rerank import CohereRerank

# setting up our LLM Settings
Settings.llm = OpenAI(model="gpt-4o-mini")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-large")

In [4]:
import os
from pathlib import Path
from glob import glob

In [5]:
pdf_path = glob("./pdf/*.pdf")
len(pdf_path)

3

In [6]:
from llama_parse import LlamaParse

parser = LlamaParse(
    result_type="markdown",
    verbose=True,
    num_workers=len(pdf_path)
)

In [7]:
# parse PDF
documents = parser.load_data(pdf_path)

Parsing files:   0%|          | 0/3 [00:00<?, ?it/s]

Parsing files: 100%|██████████| 3/3 [03:21<00:00, 67.13s/it]


In [8]:
len(documents)

238

In [9]:
node_parser = MarkdownElementNodeParser(
    llm=OpenAI(model="gpt-4o-mini"),
    num_worker=len(pdf_path)
)
nodes = node_parser.get_nodes_from_documents(documents)

0it [00:00, ?it/s]
2it [00:00, 14794.72it/s]
0it [00:00, ?it/s]
2it [00:00, 16008.79it/s]
1it [00:00, 6364.65it/s]
1it [00:00, 8630.26it/s]
1it [00:00, 6944.21it/s]
2it [00:00, 14217.98it/s]
1it [00:00, 8050.49it/s]
0it [00:00, ?it/s]
3it [00:00, 10727.12it/s]
18it [00:00, 27513.66it/s]
6it [00:00, 61832.49it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
1it [00:00, 12787.51it/s]
1it [00:00, 8943.08it/s]
4it [00:00, 33354.31it/s]
1it [00:00, 7358.43it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
1it [00:00, 10205.12it/s]
2it [00:00, 15169.27it/s]
6it [00:00, 20901.85it/s]
7it [00:00, 42987.01it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
1it [00:00, 11915.64it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
1it [00:00, 8490.49it/s]
1it [00:00, 2123.70it/s]
1it [00:00, 12945.38it/s]
4it [00:00, 32326.04it/s]
3it [00:00, 25837.60it/s]
2it [00:00, 4299.65it/s]
4it [00:00, 8401

In [11]:
base_nodes, objects = node_parser.get_nodes_and_objects(nodes)
recursive_index = VectorStoreIndex(nodes=base_nodes + objects)

In [12]:
reranker = CohereRerank(top_n=5)

In [13]:
query_engine = recursive_index.as_query_engine(
    similarity_top_k=5,
    node_postprocessor=[reranker],
    verbose=True
)

#### page_nodesを加える

In [88]:
from copy import deepcopy
from llama_index.core.schema import TextNode

def get_page_nodes(docs, separator="\n---\n"):
    """Split each document into page node, by separator."""
    nodes = []
    for doc in docs:
        doc_chunks = doc.text.split(separator)
        for doc_chunk in doc_chunks:
            node = TextNode(
                text=doc_chunk,
                metadata=deepcopy(doc.metadata)
            )
            nodes.append(node)
    return nodes

page_nodes = get_page_nodes(documents)

recursive_index_page = VectorStoreIndex(nodes=base_nodes + objects + page_nodes)
recursive_pages_query_engine = recursive_index_page.as_query_engine(
    similarity_top_k=5,
    node_postprocessors=[reranker],
    verbose=True
)


🍩 https://wandb.ai/y-hiroki-rad/llama-parse-test/r/call/0194b54a-4b51-74f1-a9e6-c205f22361f4
🍩 https://wandb.ai/y-hiroki-rad/llama-parse-test/r/call/0194b54a-edd2-7de2-a66d-2afaa3f28c10
🍩 https://wandb.ai/y-hiroki-rad/llama-parse-test/r/call/0194b54b-d1b2-7ca3-8dbd-6cbcb7c19e29
🍩 https://wandb.ai/y-hiroki-rad/llama-parse-test/r/call/0194b54d-c100-7d11-ab33-f46268353bc3
🍩 https://wandb.ai/y-hiroki-rad/llama-parse-test/r/call/0194b54e-d9f9-7303-a325-3f040f5c3964
🍩 https://wandb.ai/y-hiroki-rad/llama-parse-test/r/call/0194b54f-76c6-7eb3-a335-31456c50b11f
🍩 https://wandb.ai/y-hiroki-rad/llama-parse-test/r/call/0194b550-3e69-7802-9b76-8f5fc42bd202
🍩 https://wandb.ai/y-hiroki-rad/llama-parse-test/r/call/0194b551-304e-7782-9c78-ed7d78d5a0ec
🍩 https://wandb.ai/y-hiroki-rad/llama-parse-test/r/call/0194b551-eabd-7ed3-846f-444d06a4100b
🍩 https://wandb.ai/y-hiroki-rad/llama-parse-test/r/call/0194b552-9145-7752-ad6a-6d2db73c700c
🍩 https://wandb.ai/y-hiroki-rad/llama-parse-test/r/call/0194b553-658c-

### インデックスの保存

In [84]:
recursive_index.storage_context.persist(persist_dir="./storage")

In [85]:
from llama_index.core import StorageContext, load_index_from_storage

# ストレージコンテキストの作成
storage_context = StorageContext.from_defaults(persist_dir="./storage")
# インデックスのロード
index = load_index_from_storage(storage_context)

In [89]:
# !page_nodeを追加した場合の保存とロード
recursive_index_page.storage_context.persist(persist_dir="./storage_page")

In [90]:
from llama_index.core import StorageContext, load_index_from_storage

# ストレージコンテキストの作成
storage_context = StorageContext.from_defaults(persist_dir="./storage_page")
# インデックスのロード
index_pages = load_index_from_storage(storage_context)

In [15]:
import weave

weave.init(project_name="llama-parse test")

/Users/user/Desktop/GenerativeAI_apps/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Logged in as Weights & Biases user: y-hiroki-rad.
View Weave data at https://wandb.ai/y-hiroki-rad/llama-parse-test/weave


In [16]:
@weave.op()
def get_pages_response(query):
    response = recursive_pages_query_engine.query(query)
    print(response)

In [17]:
@weave.op()
def get_response(query):
    response = query_engine.query(query)
    print(response)

### テスト

In [ ]:
"""
質問1: 4℃グループの人材育成プログラムの根幹は何ですか？
回答: サステナブル経営の根幹である『人財』に対するグループの姿勢を表すものです
"""
query = "4℃グループの人材育成プログラムの根幹は何ですか？"
print("== not page_nodes ==")
get_response(query)
print("== used page_nodes ==")
get_pages_response(query)

== not page_nodes ==


4℃グループの人材育成プログラムの根幹は、持続的に人を育てるための教育を体系化し、世代を超えて実施することにあります。このプログラムは、企業価値を高めるために多様な人材の育成に注力し、特に次世代リーダーの育成や専門スキルの向上を目指しています。また、教育を通じて期待と愛情を伝える手段として捉え、従業員の能力開発とキャリア形成を支援する環境を提供しています。
🍩 https://wandb.ai/y-hiroki-rad/llama-parse-test/r/call/0194b459-29cf-7e71-a245-1f721815e085
== used page_nodes ==
4℃グループの人材育成プログラムの根幹は、「人間尊重」を基本理念とし、多様な人財の育成に積極的に取り組むことです。これにより、企業価値を高め、持続可能な社会の実現を目指しています。また、次世代リーダーの育成や専門スキルの向上を目的とした教育プログラムを体系化し、従業員の能力開発とキャリア形成を支援しています。
🍩 https://wandb.ai/y-hiroki-rad/llama-parse-test/r/call/0194b459-489d-7650-8046-e7614ce8ffd4


In [ ]:
"""
質問2: 4℃グループが事業継続のために策定している計画にはどのようなものがありますか？
回答: 台風や大雨による店舗の営業停止、物流プロセスの寸断などのリスクに対応するため、事業継続計画を策定しています
"""
query = "4℃グループの人材育成プログラムの根幹は何ですか？"
print("== not page_nodes ==")
get_response(query)
print("== used page_nodes ==")
get_pages_response(query)

== not page_nodes ==


4℃グループは、事業継続のために第7次中期経営計画を策定しています。この計画では、環境変化への対応を前提に顧客提供価値を追求し、将来の成長基盤を構築することを目指しています。具体的には、「4℃」ブランドの価値を具現化し、高収益ビジネスモデルの再構築を図ることが中心となっています。また、ファッションジュエリーの再構築やECチャネルの拡充、OMO戦略の推進などが含まれています。さらに、第8次中期経営計画以降に向けて、アパレル事業の拡大やM&A、海外展開、新規事業の実行なども計画されています。
🍩 https://wandb.ai/y-hiroki-rad/llama-parse-test/r/call/0194b459-7e15-75d2-94e4-f28a8dd9493c
== used page_nodes ==
4℃グループは、事業継続のために第7次中期経営計画を策定しています。この計画では、「環境変化への対応」を前提に「顧客提供価値」を追求し、将来の成長基盤を構築することを目指しています。具体的な取り組みとしては、ブランド事業の価値を具現化し、高収益ビジネスモデルの再構築、ファッションジュエリーの再構築、ECチャネルの拡充、OMO戦略の推進などが挙げられます。また、アパレル事業においては、店舗の出店拡大や安定的な配当、顧客化の深耕を図る計画も含まれています。これらの取り組みを通じて、持続的な成長を実現することを目指しています。
🍩 https://wandb.ai/y-hiroki-rad/llama-parse-test/r/call/0194b459-9c89-7353-b958-5af4556e6cbc


In [ ]:
"""
質問3: 4℃グループが人的資本経営の実践のために推進していることは何ですか？
回答: 従業員のwell-beingを重視し、健康経営、ワークライフバランス、多様性尊重などを推進しています
"""
query = "4℃グループの人材育成プログラムの根幹は何ですか？"
print("== not page_nodes ==")
get_response(query)
print("== used page_nodes ==")
get_pages_response(query)

== not page_nodes ==
Retrieval entering 61836f03-00da-4d53-bc86-2366bfef784f: TextNode
Retrieving from object TextNode with query 4℃グループが人的資本経営の実践のために推進していることは何ですか？
4℃グループは人的資本経営の実践のために、以下の取り組みを推進しています。

1. **人財育成と教育プログラム**: 次世代リーダーの育成を目的とした「若手経営者懇談会」や、専門スキル向上を目指す「アドバンスト・スクール」、女性の活躍を支援するプログラムを実施しています。

2. **健康とワークライフバランスの向上**: 従業員の健康を重視し、テレワーク制度の導入や残業時間の削減、男性育児休暇の取得促進に取り組んでいます。

3. **多様性の尊重**: 従業員が身体的、精神的、社会的に満たされた状態で働ける環境を目指し、多様性や個性を尊重するアプローチを推進しています。

4. **女性管理職比率の目標設定**: 2030年度に女性管理職比率を40.0％にすることを目指し、勉強会や社外セミナーへの参加を通じて女性のキャリアアップを支援しています。

5. **人的資本の評価基準の導入**: ESGやウェルビーイングを軸とした評価基準を施策に組み込み、組織文化の醸成を図っています。

これらの取り組みを通じて、企業価値の向上と持続可能な社会の実現を目指しています。
🍩 https://wandb.ai/y-hiroki-rad/llama-parse-test/r/call/0194b459-c676-77f2-85fe-5e0de54dd23e
== used page_nodes ==
Retrieval entering 61836f03-00da-4d53-bc86-2366bfef784f: TextNode
Retrieving from object TextNode with query 4℃グループが人的資本経営の実践のために推進していることは何ですか？
4℃グループは、人的資本経営の実践のために、従業員の健康を重視し、テレワーク制度の導入や残業時間の削減を通じてワークライフバランスの向上に取り組んでいます。また、身体・

In [ ]:
"""
質問3: IHIグループが事業開発と技術開発で重視していることは何ですか？
回答: バリューチェーン視点での事業開発と、将来のコアとなる可能性の高い技術の獲得を重視しています
"""
query = "4℃グループの人材育成プログラムの根幹は何ですか？"
print("== not page_nodes ==")
get_response(query)
print("== used page_nodes ==")
get_pages_response(query)

== not page_nodes ==
IHIグループが事業開発と技術開発で重視しているのは、バリューチェーン視点を持った技術開発と、地域戦略を組み合わせた事業開発です。具体的には、戦略的なコア技術の育成や、国内外の社会課題の解決に向けた新たなビジネスモデルの構築が重要視されています。また、技術動向の把握や必要な技術の迅速な獲得も重要なミッションとされています。これにより、競争優位性を確保し、持続可能な社会の実現を目指しています。
🍩 https://wandb.ai/y-hiroki-rad/llama-parse-test/r/call/0194b45a-0531-79f2-a9cd-402dcd608d5b
== used page_nodes ==
IHIグループが事業開発と技術開発で重視しているのは、バリューチェーン視点を持った技術開発と、地域戦略を組み合わせたスピード感とスケール感のある事業開発です。また、社内外の技術動向を把握し、必要な技術を迅速に獲得することも重要なミッションとされています。
🍩 https://wandb.ai/y-hiroki-rad/llama-parse-test/r/call/0194b45a-208a-7953-9c67-dac44fde9ce1


In [ ]:
"""
質問1: 日産が目指すサステナビリティとは何ですか？
回答: よりクリーンで、安全で、インクルーシブな、誰もが共生できる世界の実現を推進し、真にサステナブルな企業となることを目指しています
"""
query = "4℃グループの人材育成プログラムの根幹は何ですか？"
print("== not page_nodes ==")
get_response(query)
print("== used page_nodes ==")
get_pages_response(query)

== not page_nodes ==
日産が目指すサステナビリティは、「人とクルマと自然の共生」を実現することです。具体的には、エネルギーや資源の使用効率を高め、多様性や循環を促進しながら、革新的な商品やサービスを提供することを目指しています。また、事業活動やクルマのライフサイクル全体で生じる環境への依存と負荷を自然が吸収可能なレベルに抑え、豊かな自然資産を次世代に引き継ぐことが究極のゴールです。さらに、気候変動や資源依存、大気品質と水の課題に優先的に取り組み、持続可能なモビリティ社会の発展に貢献することを重視しています。
🍩 https://wandb.ai/y-hiroki-rad/llama-parse-test/r/call/0194b45a-3595-7d83-90e4-93151e056c78
== used page_nodes ==
日産が目指すサステナビリティは、コーポレートパーパス「人々の生活を豊かに。イノベーションをドライブし続ける」に基づき、信頼される企業として革新的なクルマやサービスを提供し、すべてのステークホルダーに優れた価値を提供することです。具体的には、よりクリーンで安全、インクルーシブな社会の実現を推進し、気候変動や資源依存、大気品質・水などの環境課題に対してポジティブな影響を与えることを目指しています。また、持続可能なモビリティ社会の発展に向けて、エネルギーや資源の使用効率を高め、多様性や循環を促進することも重要な要素です。最終的には、事業活動やクルマのライフサイクル全体での環境への依存と負荷を自然が吸収可能なレベルに抑え、豊かな自然資産を次世代に引き継ぐことを目指しています。
🍩 https://wandb.ai/y-hiroki-rad/llama-parse-test/r/call/0194b45a-4ddf-7872-bd0d-8a8bd8b9a00c


In [ ]:
"""
質問3: 日産が従業員の能力開発のために行っていることは何ですか？
回答: グローバル共通のトレーニングプラットフォームの導入、重要なスキル習得の機会提供、テクノロジーを活用した学習体験の充実化、実効性のある学びの機会創出、タレントアクセラレーションプログラムの見直しと再構築などを通じて、従業員の能力開発を推進しています
"""
query = "4℃グループの人材育成プログラムの根幹は何ですか？"
print("== not page_nodes ==")
get_response(query)
print("== used page_nodes ==")
get_pages_response(query)

== not page_nodes ==
日産は従業員の能力開発のために、様々なプログラムを提供しています。具体的には、マネジメントスキル、ビジネススキル、リーダーシップ開発などの多様な研修を実施し、オンラインやオンデマンド形式での講座も拡充しています。また、キャリア形成を支援するために、上司とのキャリア面談を年3回行い、従業員の自発的な学びを促進しています。さらに、コンピテンシー評価やパフォーマンス評価を導入し、専門知識やスキルの向上を図っています。リモートワーク環境下でも、2万種類以上のe-ラーニングコンテンツを提供し、デジタル学習の基盤を強化しています。
🍩 https://wandb.ai/y-hiroki-rad/llama-parse-test/r/call/0194b45a-678d-7061-ae78-ef9ebf6e3869
== used page_nodes ==
日産は従業員の能力開発のために、以下の取り組みを行っています。

1. **能力開発プログラム**: 従業員のエンプロイアビリティを向上させるためのプログラムを実施し、社会の変革に対応できる人材を育成しています。

2. **キャリア面談**: 全従業員を対象に年3回、上司とのキャリア面談を行い、キャリア形成をサポートしています。

3. **評価制度の導入**: スキルや知識に基づく「コンピテンシー評価」と、成果に基づく「パフォーマンス評価」を導入し、評価結果を報酬に結びつけています。

4. **e-ラーニングコンテンツ**: 2万種類以上のe-ラーニングコンテンツを提供し、リモートワーク環境下でも効率的に学べる機会を整えています。

5. **日産ラーニングセンター**: 技術革新に対応するため、リーダー層の育成や次世代への技術・技能伝承を目的とした学習機会を提供しています。

6. **シニアパートナー制度**: 高い専門性と経験を持つシニア従業員が継続的に活躍できるよう、柔軟な働き方を整備しています。

7. **技術講座の提供**: マネジメントスキルやビジネススキル、リーダーシップ開発など、多様なプログラムを提供し、オンライン化やオンデマンド化を進めています。

8. **日産ソフトウェアトレーニングセンター**: ソフトウェア開発スキルを持つ技術者の育成に努め、デジタ

### documentsのセパレータのチェック

In [58]:
def count_separators(doc):
   """ドキュメント内の区切りの数を数える"""
   # text_resourceからテキストを取得
   text = doc.text_resource.text

   # 指定された区切り
   test_separators = [
       "\n---\n",
       "\n\n",
       "\f",  # フォームフィード（ページ区切り）
       "\n\n\n",
       "==========",
       "# ",      # 見出しでの区切り
       "\n# "     # 改行+見出しでの区切り
   ]

   # 各区切りの出現回数を数える
   for separator in test_separators:
       count = text.count(separator)
       print(f"Separator {repr(separator)}: {count}回")

def analyze_pdf(documents):
   """ドキュメントごとに区切りを分析"""
   for i, doc in enumerate(documents, 1):
       print(f"\nDocument {i} (ID: {doc.id_}):")
       count_separators(doc)

In [61]:
analyze_pdf(documents)


Document 1 (ID: 0d8f28d9-0ad6-43c2-acdc-7a17d2963d2f):
Separator '\n---\n': 0回
Separator '\n\n': 3回
Separator '\x0c': 0回
Separator '\n\n\n': 0回
Separator '==========': 0回
Separator '# ': 3回
Separator '\n# ': 2回

Document 2 (ID: 3253c25c-2359-44ad-b8de-f91cc86cecb8):
Separator '\n---\n': 0回
Separator '\n\n': 34回
Separator '\x0c': 0回
Separator '\n\n\n': 0回
Separator '==========': 0回
Separator '# ': 13回
Separator '\n# ': 12回

Document 3 (ID: 84a6f1a1-591a-4545-8cdf-80d036627a33):
Separator '\n---\n': 0回
Separator '\n\n': 28回
Separator '\x0c': 0回
Separator '\n\n\n': 0回
Separator '==========': 0回
Separator '# ': 9回
Separator '\n# ': 8回

Document 4 (ID: 7a0e50d3-c6ef-4008-96c3-6487479186dc):
Separator '\n---\n': 0回
Separator '\n\n': 9回
Separator '\x0c': 0回
Separator '\n\n\n': 0回
Separator '==========': 0回
Separator '# ': 4回
Separator '\n# ': 3回

Document 5 (ID: 697a035a-aac8-43cb-84b3-5a29e0323553):
Separator '\n---\n': 0回
Separator '\n\n': 9回
Separator '\x0c': 0回
Separator '\n\n\n': 0回
Sep